In [1]:
# Import required modules
%load_ext autoreload
%autoreload 2

import numpy as np
import os.path
import pickle

import argparse
import torch

import random
import numpy.random
import torch.random

from datetime import datetime

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# Define variables
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()

# 2023-05-02
# For Using GPU (CUDA)
args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2023-05-02
# For using tensorboard
args.istensorboard = False

args.total_episode = 100
args.learning_rate = 0.1 # Learning rate Alpha
args.boltzmann_tau = 5 # Temperature parameter at Boltzmann policy Tau
args.total_reward = 0
args.epsilon = 0.1

args.raplay_batch = 100 #
args.batch_size = 20

args.gridnum_x = 15
args.gridnum_y = 15
args.gridsize_x = 120 # ft
args.gridsize_y = 120 # ft

# 2023-05-02: For reproduction
args.random_seed = 202022673
random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.random.seed(args.random_seed)

# State: Pressure distribution, Oil saturation

# Action: Well placement (Coordinate of well location)
args.actions = [[(i, j) for i in range(1, args.gridnum_x+1)] for j in range(1, args.gridnum_y+1)]
args.policy = np.ones((len(args.actions), len(args.actions[0]))) / (len(args.actions) * len(args.actions[0]))

args.discount_factor = 1


In [3]:
# Define CNN Structure


In [4]:
# Implement DQN Algorithm


In [5]:
# Return & Visualize Optimization result
# Final well placement

In [ ]:
# Compare PSO Result & DQN Result